About the dataset : 
1. id     : unique id for a news article
2. tittle : the title of a news artickle
3. author : author of the news artickle
4. text   : the tect of the article, could be incomplete
5. label  : a label that maks whether the news article is real or fake

   1 : fake
   0 : real

Tentang dataset:
1. id: id unik untuk sebuah artikel berita
2. judul: judul dari sebuah artikel berita
3. penulis: penulis dari artikel berita
4. teks: teks dari artikel, bisa jadi tidak lengkap
5. label: label yang menandakan apakah artikel berita itu nyata atau palsu
   1 : fake
   0 : real

In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
print(stopwords.words('english'))

Data Preprocessing

In [ ]:
# loading the dataset
df = pd.read_csv('/kaggle/input/fake-news/train.csv')
df

In [ ]:
df.shape

In [ ]:
# counting the number of missing values
df.isnull().sum()

In [ ]:
# replacing the nun values with empty string
df = df.fillna('')

In [ ]:
df.isnull().sum()

In [ ]:
# marge author name + title
df['content'] = df['author'] + ' ' + df['title']

In [ ]:
df.head()

In [ ]:
# separating the data and label
X = df.drop(columns= 'label', axis=1)
y = df['label']

In [ ]:
X

In [ ]:
y

Steening : 

Steening is the process of reducing convert to tirs root word.

Example : 
actor, actrees, actiing  -- > act

In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-z]', ' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [ ]:
#test
stemming('Lugastyan hai i love you so Much +)+((+')

In [ ]:
df['content'] = df['content'].apply(stemming)

In [ ]:
df.sample(5)

In [ ]:
# separating the data and label
X = df['content'].values
y = df['label'].values 

In [ ]:
X.shape, y.shape

In [ ]:
X

In [ ]:
y

In [ ]:
# converting content with TFIDF

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)



In [ ]:
X

In [ ]:
print(X)

Splitting the dataset training & test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=2)

In [ ]:
X_train.shape, y_train.shape

In [ ]:
X_test.shape, y_test.shape

Training model logistic regression

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train, y_train)

Evaluation

In [ ]:
# accuracy in training data

X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, y_train)
training_data_accuracy

In [ ]:
# accuracy in test data

X_test_prediction = model.predict(X_test)
training_data_accuracy = accuracy_score(X_test_prediction, y_test)
training_data_accuracy

Making Predictive System

In [ ]:
df.loc[df['label'] == 0].index

In [ ]:
df['content'][20796]

In [ ]:
df['label'][20796]

In [ ]:
test = input()
test = [stemming(test)]
test = vectorizer.transform(test)
prediction = model.predict(test)
if (prediction[0] == 1):
    print('the news is real')
else:
    print('the news is fake')


In [ ]:
model.predict(test)